In [14]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
# import pandas.stats.moments as st
import statsmodels.api as sm # import statsmodels 
from pandas import ExcelWriter
import matplotlib.pyplot as pyplot
import scipy.stats as st
import os
import quandl as qd
from collections import defaultdict
import seaborn as sns
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return


In [15]:
# Importing data from Quandl

btc = qd.get("BITFINEX/BTCUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
btc.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

eth = qd.get("BITFINEX/ETHUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
eth.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

ltc = qd.get("BITFINEX/LTCUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
ltc.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

neo = qd.get("BITFINEX/NEOUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
neo.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

etc = qd.get("BITFINEX/ETCUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
etc.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

rrt = qd.get("BITFINEX/RRTUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
rrt.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

xmr = qd.get("BITFINEX/XMRUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
xmr.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

zec = qd.get("BITFINEX/ZECUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
zec.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

omg = qd.get("BITFINEX/OMGUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
omg.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

In [16]:
# List of Crypto Data

# BTC ETH XRP BCH LTC DASH NEM Monero IOTA ETC OMG NEO BCC LSK
# Data has Open, High, Low, Close, Volume, Marketcap

def crypto_ma_opt(crypto, fast = 1, slow = 10, show_values = False):
    
    df = crypto.copy()
    
    df['fast'] = df[['Close']].rolling(fast).mean().shift(1)
    df['slow'] = df['Close'].rolling(slow).mean().shift(1)
    
    df['return'] = df['Close']/df['Close'].shift(1) - 1
    df['signal'] = df['fast'] >= df['slow']
    
    df['signal_return'] = df['signal']*df['return'].shift(-1)
    df['cumulative_return'] = ((df['return']+1).cumprod() - 1)
    df['signal_cumulative'] = ((df['signal_return']+1).cumprod() - 1)
    
    df = df.dropna()
    
    cumulative_return = df['cumulative_return'][-1]
    signal_cumulative = df['signal_cumulative'][-1]
    
    df_graph = df[['cumulative_return','signal_cumulative']]
    df_graph.columns = ['Cumulative Return', 'Algo Cumulative Return']
    
    if show_values:
        print(str(round(cumulative_return,2)))
        print(str(round(signal_cumulative,2)))
        df_graph.plot(figsize = (20,10))
        pyplot.title('Cumulative Returns', fontsize = 30)
        pyplot.ylabel('Returns', fontsize = 20)
        pyplot.xlabel('Date', fontsize = 20)
        pyplot.tick_params(labelsize=16)
        pyplot.legend(prop={'size': 16})
    
    return signal_cumulative

def heat_map(df):
    """
    This creates our heatmap using our sharpe ratio dataframe
    """
    fig = pyplot.figure(figsize=(10, 20))
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values,cmap = pyplot.get_cmap('RdYlGn'), interpolation = 'nearest')
    ax.set_xlabel(df.columns.name, fontsize = 16)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns), fontsize = 16)
    ax.set_ylabel(df.index.name, fontsize = 16)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels(list(df.index), fontsize = 16)
    ax.set_title("SMA Returns", fontsize = 20)
    pyplot.colorbar(axim)
    pyplot.tick_params(labelsize=16)
    
def sma_opt(fast_upper, slow_upper, crypto, show_heat = True, show_returns = False, show_backtest = False):
    
    # This will create a list with 9 entries
    short_mavg_days = [days for days in np.arange(5, fast_upper, 5)]
    # This will create a list with 43 entries
    long_mavg_days = [days for days in np.arange(10, slow_upper, 5)]

    #: Create a dictionary to hold all the results of our algorithm run
    all_returns = defaultdict(dict)

    # Count the number of backtests run
    backtest_count = 0

    # This will loop and run backtests
    # Each backtest takes about 3 seconds
    for short_mavg_day in short_mavg_days:
        for long_mavg_day in long_mavg_days:
            # Only consider cases where the short is less than long.. but why this not working?
            if short_mavg_day < long_mavg_day:
                
                curr_return = crypto_ma_opt(crypto, short_mavg_day, long_mavg_day)
                
                # Keep track of how many backtests were run
                if show_backtest:
                    backtest_count += 1
                    print("Backtest " + str(backtest_count) + " completed...")
            
                #: Add the result to our dict
                all_returns[short_mavg_day][long_mavg_day] = curr_return

    if show_backtest:
        print(" ")
        print("All backtest simulations completed!")
        print(" ")

    all_returns = pd.DataFrame(all_returns)
    all_returns.index.name = "Long Moving Average Days"
    all_returns.columns.name = "Short Moving Average Days"
    
    if show_heat:
        heat_map(all_returns)
    
    short_opt = all_returns.max().idxmax()
    long_opt = all_returns.max(axis = 1).idxmax()
    ret_opt = max(all_returns.max())
    print("Optimal MA cross is: " + str(short_opt) + "-" + str(long_opt))
    print("With optimal return: " + str(round(ret_opt*100,2)) + "%")
    
    if show_returns:
        crypto_ma_opt(crypto, short_opt, long_opt, show_values = True)
    
    return all_returns

def crypto_ma_signal(crypto, fast = 1, slow = 10):
    
    df = crypto.copy()
    
    df['fast'] = df[['Close']].rolling(fast).mean().shift(1)
    df['slow'] = df['Close'].rolling(slow).mean().shift(1)
    
    df['return'] = df['Close']/df['Close'].shift(1) - 1
    df['signal'] = df['fast'] >= df['slow']
    
    return df[['Close','fast','slow','signal']]

In [17]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
btc_opt = sma_opt(35, 65, btc, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(btc, 10, 20))

IndexError: index out of bounds

In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
eth_opt = sma_opt(35, 65, eth, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(eth, 5, 30))

In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
ltc_opt = sma_opt(35, 65, ltc, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(ltc, 15, 35))

In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
neo_opt = sma_opt(20, 20, neo, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(neo, 5, 10))

In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
etc_opt = sma_opt(35, 65, etc, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(etc, 5, 25))

In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
rrt_opt = sma_opt(35, 65, rrt, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(rrt, 5, 10))

In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
xmr_opt = sma_opt(35, 65, xmr, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(xmr, 15, 20))



In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
# ZEC 5-25
# OMG
zec_opt = sma_opt(35, 65, zec, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(zec, 5, 25))



In [ ]:
# Crypto latest MAs
# BTC 10-20
# ETH 5-30
# LTC 15-35
# NEO 5-20
# ETC 5-25
# RRT 5-10
# XMR 15-20
# ZEC 5-25
# OMG 10-15
omg_opt = sma_opt(35, 65, omg, show_heat = True, show_returns = True, show_backtest = False)
print(crypto_ma_signal(omg, 5, 25))



In [ ]:
crypto_ma_signal(btc, 10, 20)